In [1]:
import ee


In [2]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWgOoFvTVV_WbRVceB30wRLEwkArfvKZfPIxACBtid8Bjx_QmZ-jQZg

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
import geemap
import ipyleaflet
import numpy as np

In [5]:
## specify areas of interest / districts
Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
#print(Districts.limit(1).getInfo())

In [6]:
## create map
Map = geemap.Map(height="350px")
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [7]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(Districts, zoom=12)

In [8]:
##Add Land use land cover dataset
WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();

## define projection for use later
WCprojection = WC.first().projection();  
print('WorldCover projection:', WCprojection.getInfo());

Map.addLayer(WorldCover, {'bands': "Map"}, "WorldCover 10m 2020 (ESA)",1);

Map.add_legend(builtin_legend='ESA_WorldCover',position='bottomleft')

WorldCover projection: {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [8.333333333333333e-05, 0, -180, 0, -8.333333333333333e-05, 84]}


In [9]:
## Add intra-urban land use dataset

ULU = ee.ImageCollection("projects/wri-datalab/urban_land_use/v1")

WRIulu = ULU.select('lulc').reduce(ee.Reducer.firstNonNull()).rename('lulc')
WRIulu = WRIulu.mask(WRIulu.mask().gt(0))
WRIroad = ULU.select('road_lulc').reduce(ee.Reducer.firstNonNull()).rename('lulc')
WRIuluwRoad = WRIulu.add(WRIroad).where(WRIroad.eq(1),6).mask(WRIulu.mask().gt(0))

ULUmaskedESA = WRIuluwRoad.updateMask(WorldCover.eq(50).Or(WorldCover.eq(60)))

ULUmaskedESA = ULUmaskedESA.reproject(
      crs= WCprojection
    )

CLASSES_7=[
  "open_space",
  "nonresidential",
  "atomistic",
  "informal_subdivision",
  "formal_subdivision",
  "housing_project",
  "road"]
COLORS_7=[
  '33A02C',
  'E31A1C',
  'FB9A99',
  'FFFF99',
  '1F78B4',
  'A6CEE3',
  '3f3f3f']  
ULU7Params = {"bands": ['lulc'], 'min': 0, 'max': 6, "opacity": 1, "palette": COLORS_7}

Map.addLayer(ULUmaskedESA,ULU7Params,"Urban Land Use 2020 (WRI) masked to WorldCover built",True)

In [10]:
## Calculate and add to map vegetation and water cover rasters

NDVIthreshold = 0.4 # decimal
NDWIthreshold = 0.3 # decimal


s2 = ee.ImageCollection("COPERNICUS/S2")

def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

def addNDWI(image):
  ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
  return image.addBands(ndwi)

withNDVI = s2.filterDate('2021-01-01', '2021-12-31').map(addNDVI).map(addNDWI);

## Make a "greenest" pixel composite and masked version.
greenest = withNDVI.qualityMosaic('NDVI');
greenestNDVI = greenest.select('NDVI')
greenestMasked = greenestNDVI.updateMask(greenestNDVI.gte(NDVIthreshold))

## Make a "bluest" pixel composite and masked version.
bluest = withNDVI.qualityMosaic('NDWI');
bluestNDWI = bluest.select('NDWI')
bluestMasked = bluestNDWI.updateMask(bluestNDWI.gte(NDWIthreshold))

## Display the result.
visParams = {"bands": ['B4', 'B3', 'B2'], max: 0.3};
#Map.addLayer(greenest, visParams, 'Greenest pixel composite',0);
ndviParams = {"bands": ['NDVI'], min: -1, max: 1, "palette": ['blue', 'white', 'green']};
#Map.addLayer(greenestNDVI, ndviParams, 'Vegetation 2020: NDVI Greenest pixel composite',0);
Map.addLayer(greenestMasked, ndviParams, 'Vegetation 2020: NDVI composite masked at >'+str(NDVIthreshold)+' (Sentinel)');

ndwiParams = {"bands": ['NDWI'], min: -1, max: 1, "palette": ['green', 'white', 'blue']};
#Map.addLayer(bluestNDWI, ndwiParams, 'Water 2020: NDWI Bluest pixel composite',0);
Map.addLayer(bluestMasked, ndwiParams, 'Water 2020: NDWI composite masked at >'+str(NDWIthreshold)+' (Sentinel)');


In [11]:
## add tree cover dataset
TML = ee.ImageCollection('projects/wri-datalab/TML')
TreeCover = TML.reduce(ee.Reducer.mean()).rename('b1')
TreePctThreshold = 10 #whole numbers - 0-100

## Reproject trees to match LULC projection


TreeCover = TreeCover.reproject(
      crs= WCprojection
    )

Map.addLayer(TreeCover.updateMask(TreeCover.gte(TreePctThreshold)),
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Tree Cover 2020 (WRI Trees in Mosaic Landscapes)',True,1)

In [12]:
#add area of interest to map
Map.addLayer(Districts,{},"Districts",True,0.5)


In [13]:
## calculations to determine tree cover by LULC class


## function to create image of means of toCount for each asClass

def getmeanbyclass(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment if you want includ only pixel that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('') #'class_count-'
                                       .cat(ee.Number(classvalue).toInt().format()))

## function to create image of count of each asClass
def getcountbyclass(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment if you want includ only pixel that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('class_count-') #'class_count-'
                                      .cat(ee.Number(classvalue).toInt().format()))


In [14]:
## create image with each WorldCover class mean as a band

asClass = WorldCover
toCount = TreeCover #greenestMasked

meanbyclass=ee.Image(getmeanbyclass(10)).addBands([
  getmeanbyclass(20),  
  getmeanbyclass(30),  
  getmeanbyclass(40),  
  getmeanbyclass(50),  
  getmeanbyclass(60),
  getmeanbyclass(70),  
    getmeanbyclass(80), 
    getmeanbyclass(90), 
    getmeanbyclass(95), 
    getmeanbyclass(100), 
])

## create image with each WorldCover class count as a band

countbyclass=ee.Image(getcountbyclass(10)).addBands([
  getcountbyclass(20),  
  getcountbyclass(30),  
  getcountbyclass(40),  
  getcountbyclass(50),  
  getcountbyclass(60),
  getcountbyclass(70),  
    getcountbyclass(80), 
    getcountbyclass(90), 
    getcountbyclass(95), 
    getcountbyclass(100), 
])

#print('meanbyclass', meanbyclass.getInfo())
#print('countbyclass', countbyclass.getInfo())
#Map.addLayer(meanbyclass.select('50'),{},"meanbyWCclass")

In [15]:
## create image with each ULU class mean as a band

asClass = ULUmaskedESA
toCount = TreeCover #greenestMasked 

meanbyclassULU=ee.Image(getmeanbyclass(0)).addBands([
  getmeanbyclass(1),  
  getmeanbyclass(2),  
  getmeanbyclass(3),  
  getmeanbyclass(4),  
  getmeanbyclass(5),
  getmeanbyclass(6) 
])

## create image with each ULU class count as a band

countbyclassULU=ee.Image(getcountbyclass(0)).addBands([
  getcountbyclass(1),  
  getcountbyclass(2),  
  getcountbyclass(3),  
  getcountbyclass(4),  
  getcountbyclass(5),
  getcountbyclass(6)
])
Map.addLayer(meanbyclassULU.select('1'),{},"meanbyULUclass")

In [16]:
## create FeatureCollection with mean of count for each class for each feature

histo=meanbyclass.reduceRegions(
  reducer= ee.Reducer.mean(), 
  collection= Districts, 
  scale= 10, 
  tileScale= 1
)

histo=countbyclass.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

histo=meanbyclassULU.reduceRegions(
  reducer= ee.Reducer.mean(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

histo=countbyclassULU.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

print('histo:', histo.limit(1).getInfo())

histo: {'type': 'FeatureCollection', 'columns': {'0': 'Float<0.0, 255.0>', '1': 'Float<0.0, 255.0>', '10': 'Float<0.0, 255.0>', '100': 'Float<0.0, 255.0>', '2': 'Float<0.0, 255.0>', '20': 'Float<0.0, 255.0>', '3': 'Float<0.0, 255.0>', '30': 'Float<0.0, 255.0>', '4': 'Float<0.0, 255.0>', '40': 'Float<0.0, 255.0>', '5': 'Float<0.0, 255.0>', '50': 'Float<0.0, 255.0>', '6': 'Float<0.0, 255.0>', '60': 'Float<0.0, 255.0>', '70': 'Float<0.0, 255.0>', '80': 'Float<0.0, 255.0>', '90': 'Float<0.0, 255.0>', '95': 'Float<0.0, 255.0>', 'Area_ha': 'Float', 'Id': 'Integer', 'Sub_City': 'String', 'Woreda': 'String', 'area_m2': 'Float', 'class_count-0': 'Long<0, 4294967295>', 'class_count-1': 'Long<0, 4294967295>', 'class_count-10': 'Long<0, 4294967295>', 'class_count-100': 'Long<0, 4294967295>', 'class_count-2': 'Long<0, 4294967295>', 'class_count-20': 'Long<0, 4294967295>', 'class_count-3': 'Long<0, 4294967295>', 'class_count-30': 'Long<0, 4294967295>', 'class_count-4': 'Long<0, 4294967295>', 'class_

In [17]:
## Define function to normalize count as percent of all pixels in each feature and create new properties with the values

def count_to_percent(feat):
    feat=ee.Feature(feat)
    hist=ee.Dictionary(feat.toDictionary(['10','20','30','40','50','60','70','80','90','95','100']))
    hist=hist.set('10',hist.get('10',0))
    hist=hist.set('20',hist.get('20',0))
    hist=hist.set('30',hist.get('30',0))
    hist=hist.set('40',hist.get('40',0))
    hist=hist.set('50',hist.get('50',0))
    hist=hist.set('60',hist.get('60',0))
    hist=hist.set('70',hist.get('70',0))
    hist=hist.set('80',hist.get('80',0))
    hist=hist.set('90',hist.get('90',0))
    hist=hist.set('95',hist.get('95',0))
    hist=hist.set('100',hist.get('100',0))
    
    def pct_hist(k,v):
        # convert whole number (0-100) to decimal percent (0-1)
        return ee.Number(v).multiply(ee.Number(0.01))
    
    pcts = hist.map(pct_hist)
    
    histC=ee.Dictionary(feat.toDictionary(['class_count-10','class_count-20','class_count-30','class_count-40','class_count-50','class_count-60','class_count-70','class_count-80','class_count-90','class_count-95','class_count-100']))
    histC=histC.set('10',histC.get('class_count-10',0))
    histC=histC.set('20',histC.get('class_count-v20',0))
    histC=histC.set('30',histC.get('class_count-30',0))
    histC=histC.set('40',histC.get('class_count-40',0))
    histC=histC.set('50',histC.get('class_count-50',0))
    histC=histC.set('60',histC.get('class_count-60',0))
    histC=histC.set('70',histC.get('class_count-70',0))
    histC=histC.set('80',histC.get('class_count-80',0))
    histC=histC.set('90',histC.get('class_count-90',0))
    histC=histC.set('95',histC.get('class_count-95',0))
    histC=histC.set('100',histC.get('class_count-100',0))
    
    def area_hist(k,v):
        # convert 10m pixel count of class to KM2 of class
        return ee.Number(v).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
    
    classAreas = histC.map(area_hist)
    
    area=feat.area().multiply(0.000001)

    totalPixels=hist.values()
    
    histULU=ee.Dictionary(feat.toDictionary(['0','1','2','3','4','5','6']))
    histULU=histULU.set('0',histULU.get('0',0))
    histULU=histULU.set('1',histULU.get('1',0))
    histULU=histULU.set('2',histULU.get('2',0))
    histULU=histULU.set('3',histULU.get('3',0))
    histULU=histULU.set('4',histULU.get('4',0))
    histULU=histULU.set('5',histULU.get('5',0))
    histULU=histULU.set('6',histULU.get('6',0))
    
    pctsULU = histULU.map(pct_hist)
        
    histULUc=ee.Dictionary(feat.toDictionary(['class_count-0','class_count-1','class_count-2','class_count-3','class_count-4','class_count-5','class_count-6']))
    histULUc=histULUc.set('0',histULUc.get('class_count-0',0))
    histULUc=histULUc.set('1',histULUc.get('class_count-1',0))
    histULUc=histULUc.set('2',histULUc.get('class_count-2',0))
    histULUc=histULUc.set('3',histULUc.get('class_count-3',0))
    histULUc=histULUc.set('4',histULUc.get('class_count-4',0))
    histULUc=histULUc.set('5',histULUc.get('class_count-5',0))
    histULUc=histULUc.set('6',histULUc.get('class_count-6',0))

    classAreasULU = histULUc.map(area_hist)

    district = feat.getString("Sub_City").cat(feat.getString("Woreda"))
    
    return feat.set({
        'LC10treepct': pcts.getNumber('10'),
        'LC20treepct': pcts.getNumber('20'),
        'LC30treepct': pcts.getNumber('30'),
        'LC40treepct': pcts.getNumber('40'),
        'LC50treepct': pcts.getNumber('50'),
        'LC60treepct': pcts.getNumber('60'),
        'LC70treepct': pcts.getNumber('70'),
        'LC80treepct': pcts.getNumber('80'),
        'LC90treepct': pcts.getNumber('90'),
        'LC95treepct': pcts.getNumber('95'),
        'LC100treepct': pcts.getNumber('100'),
        'TotalareaKM2': area,
        'TotalPixels': totalPixels,
        'District': district,
        'LC10areaKM2': classAreas.getNumber('10'),
        'LC20areaKM2': classAreas.getNumber('20'),
        'LC30areaKM2': classAreas.getNumber('30'),
        'LC40areaKM2': classAreas.getNumber('40'),
        'LC50areaKM2': classAreas.getNumber('50'),
        'LC60areaKM2': classAreas.getNumber('60'),
        'LC70areaKM2': classAreas.getNumber('70'),
        'LC80areaKM2': classAreas.getNumber('80'),
        'LC90areaKM2': classAreas.getNumber('90'),
        'LC95areaKM2': classAreas.getNumber('95'),
        'LC100areaKM2': classAreas.getNumber('100'),
        'ULU0treepct': pctsULU.getNumber('0'),
        'ULU1treepct': pctsULU.getNumber('1'),
        'ULU2treepct': pctsULU.getNumber('2'),
        'ULU3treepct': pctsULU.getNumber('3'),
        'ULU4treepct': pctsULU.getNumber('4'),
        'ULU5treepct': pctsULU.getNumber('5'),
        'ULU6treepct': pctsULU.getNumber('6'),
        'ULU0areaKM2': classAreasULU.getNumber('0'),
        'ULU1areaKM2': classAreasULU.getNumber('1'),
        'ULU2areaKM2': classAreasULU.getNumber('2'),
        'ULU3areaKM2': classAreasULU.getNumber('3'),
        'ULU4areaKM2': classAreasULU.getNumber('4'),
        'ULU5areaKM2': classAreasULU.getNumber('5'),
        'ULU6areaKM2': classAreasULU.getNumber('6'),
    })

In [18]:
## update FeatureCollection with percents

tree_pcts=histo.map(count_to_percent)

print('Tree cover % by Land Cover class for Districts',tree_pcts.limit(1).getInfo());

Tree cover % by Land Cover class for Districts {'type': 'FeatureCollection', 'columns': {'0': 'Float<0.0, 255.0>', '1': 'Float<0.0, 255.0>', '10': 'Float<0.0, 255.0>', '100': 'Float<0.0, 255.0>', '2': 'Float<0.0, 255.0>', '20': 'Float<0.0, 255.0>', '3': 'Float<0.0, 255.0>', '30': 'Float<0.0, 255.0>', '4': 'Float<0.0, 255.0>', '40': 'Float<0.0, 255.0>', '5': 'Float<0.0, 255.0>', '50': 'Float<0.0, 255.0>', '6': 'Float<0.0, 255.0>', '60': 'Float<0.0, 255.0>', '70': 'Float<0.0, 255.0>', '80': 'Float<0.0, 255.0>', '90': 'Float<0.0, 255.0>', '95': 'Float<0.0, 255.0>', 'Area_ha': 'Float', 'District': 'String', 'Id': 'Integer', 'LC100areaKM2': 'Number', 'LC100treepct': 'Number', 'LC10areaKM2': 'Number', 'LC10treepct': 'Number', 'LC20areaKM2': 'Number', 'LC20treepct': 'Number', 'LC30areaKM2': 'Number', 'LC30treepct': 'Number', 'LC40areaKM2': 'Number', 'LC40treepct': 'Number', 'LC50areaKM2': 'Number', 'LC50treepct': 'Number', 'LC60areaKM2': 'Number', 'LC60treepct': 'Number', 'LC70areaKM2': 'Numb

In [19]:
## render on map percent tree cover by class from feature collection

empty = ee.Image().byte()
Tpctfills = empty.paint(**{'featureCollection': tree_pcts,'color': 'ULU1treepct'})

fillspalette = ['red', 'green']
Map.addLayer(Tpctfills, {'palette': fillspalette,'min':0,'max':0.15}, '% of Non-res area with tree cover', True, 0.65)
Map

Map(center=[8.98091786068599, 38.78520904303285], controls=(WidgetControl(options=['position', 'transparent_bg…

In [20]:
## select properties to keep, sort features and create data frame to display properties
tree_pctsSort = tree_pcts.select(['Woreda','Sub_City','District','TotalareaKM2','LC10areaKM2','LC20areaKM2','LC30areaKM2','LC40areaKM2','LC50areaKM2','LC60areaKM2','LC70areaKM2','LC80areaKM2','LC90areaKM2','LC95areaKM2','LC100areaKM2','LC10treepct','LC20treepct','LC30treepct','LC40treepct','LC50treepct','LC60treepct','LC70treepct','LC80treepct','LC90treepct','LC95treepct','LC100treepct','ULU0treepct','ULU1treepct','ULU2treepct','ULU3treepct','ULU4treepct','ULU5treepct','ULU6treepct','ULU0areaKM2','ULU1areaKM2','ULU2areaKM2','ULU3areaKM2','ULU4areaKM2','ULU5areaKM2','ULU6areaKM2']).sort('ULU1treepct', False) #
#print('Tree cover sorted version',tree_pctsSort.limit(1).getInfo());

In [21]:
df = geemap.ee_to_pandas(tree_pctsSort)
df

,LC90areaKM2,LC95treepct,LC95areaKM2,LC50areaKM2,LC60areaKM2,TotalareaKM2,LC70treepct,LC70areaKM2,LC90treepct,Sub_City,...,ULU5areaKM2,LC20areaKM2,LC100areaKM2,District,LC10areaKM2,LC30treepct,ULU3areaKM2,ULU6areaKM2,ULU0areaKM2,ULU2areaKM2
0,0.0000,0,0,0.6138,0.2510,1.374909,0,0,0.000000,Gulele,...,0.0082,0,0,Gulele05,0.3184,0.094277,0.1514,0.0757,0.0644,0.5376
1,0.0000,0,0,1.1527,0.7036,6.846928,0,0,0.000000,Yeka,...,0.0217,0,0,Yeka05,2.7491,0.226432,0.2956,0.1533,0.3930,0.4246
2,0.0000,0,0,0.6399,0.2967,1.164112,0,0,0.000000,Lideta,...,0.0022,0,0,Lideta02,0.0509,0.123346,0.0134,0.1187,0.1081,0.4359
3,0.0000,0,0,0.4555,0.2356,0.717098,0,0,0.000000,Kolfe_Keranio,...,0.0037,0,0,Kolfe_Keranio12,0.0216,0.165926,0.0171,0.1185,0.0250,0.4067
4,0.0000,0,0,1.3972,0.5245,1.983953,0,0,0.000000,Gulele,...,0.0059,0,0,Gulele09,0.0589,0.260126,0.1444,0.2630,0.0702,0.9825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0.0000,0,0,7.0449,7.3724,27.654482,0,0,0.000000,Bole,...,2.0986,0,0,Bole10,0.1603,0.104121,2.4636,2.2497,4.9037,0.3715
112,0.0042,0,0,3.4274,5.9581,27.260199,0,0,0.038095,Bole,...,0.2631,0,0,Bole11,0.4150,0.046372,0.3915,0.8939,4.5369,0.7175
113,0.0000,0,0,0.8253,2.6109,12.839059,0,0,0.000000,Akaki Akality,...,0.0200,0,0,Akaki Akality10,0.0673,0.023061,0.0160,0.1872,2.7196,0.0464
114,0.0000,0,0,0.3700,0.3968,0.770569,0,0,0.000000,Addis Ketema,...,0.0032,0,0,Addis Ketema01,0.0010,0.000000,0.0429,0.1302,0.0523,0.1456


In [22]:
## display features in chart

import geemap.chart as chart

xProperty = "District" #,"Woreda"
yProperties = ['ULU1treepct'] # ,'LC50areaKM2'

options = {
    'xlabel': "District",
    'ylabel': "Tree Cover share",
    "legend_location": "top-right",
    "height": "500px",
}

chart.feature_byFeature(tree_pctsSort, xProperty, yProperties, **options)

In [23]:
# Download attribute table as a CSV

url = tree_pctsSort.getDownloadURL(
    filetype="csv",
    #selectors=['NAME', 'ALAND', 'REGION', 'STATEFP', 'STUSPS'],
    filename="tree_pctsSort",
)
print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/cb0a6629484da7269a4617039017b983-1347560ff3d4bef77bc2fa35e6197c87:getFeatures


In [24]:
# Download data as shapefile to Google Drive

# function for converting GeometryCollection to Polygon/MultiPolygon
def filter_polygons(ftr):
    geometries = ftr.geometry().geometries()
    geometries = geometries.map(
        lambda geo: ee.Feature(ee.Geometry(geo)).set('geoType', ee.Geometry(geo).type())
    )

    polygons = (
        ee.FeatureCollection(geometries)
        .filter(ee.Filter.eq('geoType', 'Polygon'))
        .geometry()
    )
    return ee.Feature(polygons).copyProperties(ftr)

tree_pctsSortSHP = tree_pctsSort.map(filter_polygons)

col_names = tree_pctsSort.first().propertyNames().getInfo()
print("Column names: ", col_names)

url = tree_pctsSortSHP.getDownloadURL("shp", col_names, 'tree_pctsSortSHP')
print(url)

# Set configuration parameters for output vector
task_config = {
    #'folder': 'gee-data',  # output Google Drive folder
    'fileFormat': 'SHP',
    #'selectors': col_names,  # a list of properties/attributes to be exported
}

print('Exporting {}'.format(tree_pctsSort))
task = ee.batch.Export.table.toDrive(tree_pctsSort, 'AddisTreePctsbyLULC', **task_config)
task.start()

Column names:  ['system:index', 'LC90areaKM2', 'LC95treepct', 'LC95areaKM2', 'LC50areaKM2', 'LC60areaKM2', 'TotalareaKM2', 'LC70treepct', 'LC70areaKM2', 'LC90treepct', 'Sub_City', 'LC80areaKM2', 'LC80treepct', 'ULU4treepct', 'ULU1areaKM2', 'ULU5treepct', 'LC20treepct', 'LC10treepct', 'LC100treepct', 'ULU6treepct', 'ULU2treepct', 'ULU3treepct', 'LC40treepct', 'LC50treepct', 'LC60treepct', 'LC40areaKM2', 'Woreda', 'LC30areaKM2', 'ULU0treepct', 'ULU1treepct', 'ULU4areaKM2', 'ULU5areaKM2', 'LC20areaKM2', 'LC100areaKM2', 'District', 'LC10areaKM2', 'LC30treepct', 'ULU3areaKM2', 'ULU6areaKM2', 'ULU0areaKM2', 'ULU2areaKM2']
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/a0b4fd4600cc72efa9592337d42c321e-1d0eef9c98c026dc26e13891f13e7153:getFeatures


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
# Download ee.Image for Land cover and tree cover as GeoTIFF

geemap.ee_export_image(
    WorldCover, filename="ESAWorldCover.tif", scale=10, region=Districts.geometry(), file_per_band=False
)

geemap.ee_export_image(
    TreeCover, filename="TMLTreeCover.tif", scale=10, region=Districts.geometry(), file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Eric.Mackres.INTERNAL\OneDrive - World Resources Institute\Python Scripts\ESAWorldCover.tif
Generating URL ...
An error occurred while downloading.
Total request size (79440993 bytes) must be less than or equal to 50331648 bytes.


In [26]:
# OR if filesize is too large for standard download - Export an ee.Image to Google Drive as TIF

geemap.ee_export_image_to_drive(
    TreeCover, description='TMLTreeCover', folder='EEexport', region=Districts.geometry(), scale=10
)

Exporting TMLTreeCover ...
